# Обычный уровень

### 1

In [15]:
import requests

def get_exchange_rate(base_currency, target_currency):
    base_currency = base_currency.lower()
    target_currency = target_currency.lower()
    url = f"https://cdn.jsdelivr.net/npm/@fawazahmed0/currency-api@latest/v1/currencies/{base_currency}.json"
    
    try:
        response = requests.get(url)
        response.raise_for_status()  # проверка на успешность запроса
        data = response.json()
        
        if target_currency in data[base_currency]:
            return data[base_currency][target_currency]
        else:
            raise ValueError(f"Валюта {target_currency} не найдена в данных API")
    except requests.exceptions.RequestException as e:
        print(f"Ошибка при запросе курса обмена: {e}")
        return None
    
def convert_currency(base_currency, target_currency, amount):
    exchange_rate = get_exchange_rate(base_currency,target_currency)
    if exchange_rate is None:
        raise ValueError(f"Невозможно получить курс обмена из {base_currency} в {target_currency}")
    convert_amount = amount*exchange_rate
    return round(convert_amount,2)

In [16]:
base_currency = "USD"  # Валюта, из которой нужно конвертировать
target_currency = "eur"  # Валюта, в которую нужно конвертировать
exchange_rate = get_exchange_rate(base_currency, target_currency)

print(f"Текущий курс {base_currency} к {target_currency}: {exchange_rate}")
# Вывод: Текущий курс USD к EUR: 0.9189

amount = 20 # Вводим кол-во базовой валюты, в данном примере - 20 USD
converted_amount = convert_currency(base_currency, target_currency, amount)

print(f"{amount} {base_currency} равно {converted_amount} {target_currency}")
# 20 USD равно 18.38 EUR

Текущий курс USD к eur: 0.92743523
20 USD равно 18.55 eur


In [14]:
print(f"Текущий курс {base_currency} к {target_currency}: {exchange_rate}")
# Вывод: Текущий курс USD к EUR: 0.9189

Текущий курс USD к eur: 0.92743523


### 2 

In [21]:
response = process_user_data()

In [20]:
import requests

def process_user_data():
    base_url = "https://jsonplaceholder.typicode.com"
    webhook_url = 'https://webhook.site/a736cf89-cc35-4527-bd74-e0ab2b0067b2'

    # Получаем данные о пользователях
    users_response = requests.get(f"{base_url}/users")
    users_response.raise_for_status()
    users = users_response.json()

    # Получаем все посты
    posts_response = requests.get(f"{base_url}/posts")
    posts_response.raise_for_status()
    posts = posts_response.json()

    # Получаем все комментарии
    comments_response = requests.get(f"{base_url}/comments")
    comments_response.raise_for_status()
    comments = comments_response.json()

    # Если данные пустые, выведем сообщение для диагностики
    if not users or not posts or not comments:
        print("Ошибка: один или несколько запросов вернули пустые данные.")
        return None

    # Подсчет количества постов и комментариев для каждого пользователя
    statistics = []
    for user in users:
        user_id = user['id']
        username = user['username']
        email = user['email']

        # Подсчет постов пользователя
        user_posts = [post for post in posts if post['userId'] == user_id]
        post_count = len(user_posts)

        # Подсчет комментариев пользователя
        user_comments = [comment for comment in comments if comment['email'] == email]
        comment_count = len(user_comments)

        # Проверка для отладки

        # Добавляем данные в итоговую статистику
        statistics.append({
            "id": user_id,
            "username": username,
            "email": email,
            "posts": post_count,
            "comments": comment_count
        })

    # Подготовка данных для отправки на вебхук
    payload = {"statistics": statistics}

    # Отправка POST-запроса на вебхук с JSON данными
    response = requests.post(webhook_url, json=payload)
    response.raise_for_status()  # Проверка на успешную отправку

    return response  # Возвращаем объект ответа запроса


# Продвинутый уровень 

### 1

In [23]:
import time

def time_decorator(func):
    def wrapper():
        start_time = time.time()
        result = func()
        end_time = time.time()
        
        elapsed_time = round(end_time - start_time)
        print(elapsed_time)
        return result 
    return wrapper


In [25]:
@time_decorator
def sleep_1_sec():
    time.sleep(3)
    print("function")
    return 25


result = sleep_1_sec()
# Выведет
# function
# 1

print(result)  # Вывод: 25

function
3
25


### 2

In [1]:
import datetime
import inspect
def logging_decorator(logger):
    def decortor(func):
        def wrapper(*args,**kwargs):
            call_args = inspect.getcallargs(func,*args,**kwargs)
            call_time = datetime.datetime.now()
            result = func(*args,**kwargs)
            log = {
                'name': func.__name__,
                'arguments': call_args,
                'call_time': call_time,
                'result': result
            }
            logger.append(log)
            return result
        return wrapper
    return decortor
            

### 3

In [4]:
import time
from functools import wraps

def cache_results(func):
    cache = {}
    @wraps(func)
    def wrapped(*args,**kwargs):
        cache_key = (args, frozenset(kwargs.items()))
        if cache_key in cache:
            print('Результат взят из кэша')
            return cache[cache_key]
        else:
            start_time = time.time()
            result = func(*args,**kwargs)
            end_time = time.time()
            cache[cache_key]=result
            elapsed_time = int(end_time-start_time)
            print(f'Выполнено за {elapsed_time} секунды')
            return result
    return wrapped